This week we will be looking at Enterococcus levels in the Hudson River, using data from the organiation Riverkeeper (http://www.riverkeeper.org/).

Background: Enterococcus is a fecal indicating bacteria that lives in the intestines of humans and other warm-blooded animals. Enterococcus (“Entero”) counts are useful as a water quality indicator due to their abundance in human sewage, correlation with many human pathogens and low abundance in sewage free environments. The United States Environmental Protection Agency (EPA) reports Entero counts as colonies (or cells) per 100 ml of water.
Riverkeeper has based its assessment of acceptable water quality on the 2012 Federal Recreational Water Quality Criteria from the US EPA. Unacceptable water is based on an illness rate of 32 per 1000 swimmers.
The federal standard for unacceptable water quality is a single sample value of greater than 110 Enterococcus/100mL, or five or more samples with a geometric mean (a weighted average) greater than 30 Enterococcus/100mL.
Data: I have provided the data on our github page, in the folder https://github.com/jlaurito/CUNY_IS608/blob/master/lecture4/data. I have not cleaned it – you need to do so.

This assignment must be done in python. It must be done using the ‘bokeh’, 'seaborn', or 'pandas' package. You may turn in either a .py file or an ipython notebook file.

Questions:

1. Create lists & graphs of the best and worst places to swim in the dataset.

2. The testing of water quality can be sporadic. Which sites have been tested most regularly? Which ones have long gaps between tests? Pick out 5-10 sites and visually compare how regularly their water quality is tested.

3. Is there a relationship between the amount of rain and water quality?  Show this relationship graphically. If you can, estimate the effect of rain on quality at different sites and create a visualization to compare them.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_notebook
from bokeh.charts import Scatter, Bar, output_file, show
 
df = pd.DataFrame.from_csv('https://raw.githubusercontent.com/jlaurito/CUNY_IS608/master/lecture4/data/riverkeeper_data_2013.csv')
df.reset_index(level=0, inplace=True)

In [2]:
df['Site'] = df['Site'].astype('category')
df['Date'] = pd.to_datetime(df['Date'])
df['EnteroCount'] = df['EnteroCount'].str.replace('>', '').str.replace('<', '')
df['EnteroCount'] = df['EnteroCount'].astype('float').fillna(0.0)
summary = pd.DataFrame(df.groupby('Site').describe().unstack())

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
x = pd.DataFrame(summary['EnteroCount', 'mean']) 
x.reset_index(level=0, inplace=True)
x.columns = x.columns.droplevel()
x.columns = ['Site', 'EnteroCount - mean']
x['Site'] = x['Site'].astype('string')
x = x.sort_values(['EnteroCount - mean'], ascending=False )
top10 = pd.DataFrame(x.head(10))
top10['Value'] = "Top 10"
bottom10 = pd.DataFrame(x.tail(10))
bottom10['Value'] = "Bottom 10"
frames = [top10, bottom10]
result = pd.concat(frames)

###### 1. The below graph illustrates the best and worst places to swim. 

In [5]:
scatter = Scatter(result, x='Site' , y='EnteroCount - mean', color = 'Value',
                  title="Top 10 and Bottom 10 Average EnteroCount", 
                  xlabel="Location", ylabel="Average EnteroCount", legend='top_right')
show(scatter)

The high average counts of Gowanus Canal is well know and I have personally seen the oily dark water of the Canal, which was not inviting for a swim. Additionally, most of the high average sites are near the very end of the Hudson River which means the colonies have collected in count along the journey of the Hudson river. 

##### Gowanus Canal
<img src="http://assets.nydailynews.com/polopoly_fs/1.1810338.1401393274%21/img/httpImage/image.jpg_gen/derivatives/article_970/gowanus-canal-brooklyn.jpg" width="400" height="100" >

The lowest average counts of EnteroCount are locations are much further up the Hudson River. Also, I would hope that Poughkeepsie Drinking Water Intake would have very low counts for the sake of those drinking this water.  

###### 2a. The testing of water quality can be sporadic. Surprisingly, we see that sites that are tested more regularly are not necessarly the ones that have the highest average EnteroCount in this time period.

In [6]:
x = pd.DataFrame(summary['SampleCount', 'count']) 
x.reset_index(level=0, inplace=True)
x.columns = x.columns.droplevel()
x.columns = ['Site', 'Sample Count']
x['Site'] = x['Site'].astype('string')
x = x.sort_values(['Sample Count'], ascending=False )
top10 = pd.DataFrame(x.head(10))

In [7]:
bar = Bar(top10, 'Site', values='Sample Count',
          title="Top 10 Sampled Sites", xlabel="Location", 
          ylabel="Count", legend='top_left')

bar.legend.orientation = "horizontal"
bar.legend.background_fill_alpha = 0.5

show(bar)

It does seem appropriate that Newtown Creek- Metropolitan Ave would be tested often and have high average EnteroCount since its near a waster treatment plant in New York City.

<img src="http://urbanomnibus.net/redux/wp-content/uploads/2009/08/newtown_esto.jpg" width="400" height="100">

However, the Piermont Pier testing seems unusual but the high amount of testing may be due to its natural marshes and delicate ecological system.

<img src="http://static.panoramio.com/photos/large/61782884.jpg" width="400" height="100" >

###### 2b. Which ones have long gaps between tests? Pick out 5-10 sites and visually compare how regularly their water quality is tested.

In [8]:
dfDateSorted = df.sort_values(['Site','Date'], ascending = [False, True])
#http://stackoverflow.com/a/25378646 - very helpful for the below time delta
deltaTime = lambda x: (x - x.shift(1))  
dfDateSorted['Days between Samples'] = dfDateSorted.groupby('Site')['Date'].apply(deltaTime)
dfDateSorted = dfDateSorted.ix[:,['Site','Days between Samples']]
dfDateSorted['Days between Samples']  = dfDateSorted['Days between Samples'] / np.timedelta64(1, 'D')
result = dfDateSorted.groupby('Site')['Days between Samples'].agg(['sum', 'mean', 'count', 'max', 'std'])

In [9]:
results = result.sort_values('mean', ascending = True)
results = results.head(10)
results.reset_index(level=0, inplace=True)
results = pd.melt(results, id_vars=['Site'], var_name='stat')

In [10]:
scatter = Scatter(results, x='Site' , y='value', color = 'stat',
                  title="Top 10 Sites with Highest Average Days between Samples", 
                  xlabel="Location", ylabel="Days between Samples Stats", legend='top_right')
show(scatter)

The site with the most average days between samples is Kingsland Pt. Park - Pocantico River. It may be that because it is only a park area without swimming so they may not test the water often.

##### Kingsland Pt. Park - Pocantico River
<img src="http://www.scenichudson.org/sites/default/files/u1/sleepypark_450.jpg" width="400" height="100" >

###### 3. Is there a relationship between the amount of rain and water quality? Show this relationship graphically. If you can, estimate the effect of rain on quality at different sites and create a visualization to compare them.

In [11]:
dfRainWater = df.sort_values(['Site','Date'], ascending = [False, True])
dfRainWater = dfRainWater.set_index(pd.DatetimeIndex(dfRainWater['Date']))
dfRainWater = dfRainWater.groupby([pd.TimeGrouper('6M', closed = 'left'), dfRainWater['Site']]).mean()

In [12]:
dfRainWater = dfRainWater[np.isfinite(dfRainWater['EnteroCount'])]
dfRainWater = dfRainWater.reset_index()
dfRainWaterCorr = dfRainWater.groupby('Site').corr()
dfRainWaterCorr = dfRainWaterCorr.reset_index()

In [13]:
dfRainWaterCorr = dfRainWaterCorr[(dfRainWaterCorr['level_1'] == 'FourDayRainTotal')]
dfRainWaterCorr = dfRainWaterCorr.drop(['level_1','FourDayRainTotal','SampleCount'], axis = 1)
dfRainWaterCorr.columns = ['Site','Correlation of EnteroCount to FourDayRainTotal']

In [14]:
dfRainWaterCorr = dfRainWaterCorr.sort_values('Correlation of EnteroCount to FourDayRainTotal')
dfRainWaterCorr = pd.DataFrame(dfRainWaterCorr)

In [15]:
scatter = Scatter(dfRainWaterCorr, x = 'Site', y = 'Correlation of EnteroCount to FourDayRainTotal',
                 plot_width=1000, plot_height=500)
show(scatter)

Looking at the above graph we can see the correlations between the average FourDayRainTotal or Rain fall and average EnteroCount for six month periods. The correlations for each site are mostly positive with a few regions having a negative correlation. In general, I believe we can safely assume that there is a positive correlation between rainfall and Enterococcus. As rainfall increases we would expect an increase in Enterococcus. This does match research that shows that increases in rainfall increases the counts of microbes in water, this is related to the runoff caused by increased rainfall.[1]

[1]: Zhang, Wang, Fan, Gao, & Ju. (2013). Effects of rainfall on microbial water quality on Qingdao No. 1 Bathing Beach, China. Marine Pollution Bulletin, 66(1-2), 185-190.